# Create polygons from polygon vertices

MASIE region vertices for this example should be [accesible here](ftp://sidads.colorado.edu/DATASETS/NOAA/G02186/ancillary/MASIE_regions_polygon_vertices.xlsx) (Note - at time of developing this, this link appears to have rotted...)

1. Read in vertices

In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

In [2]:
masie_df = pd.read_excel("ancillary/MASIE_regions_polygon_vertices.xls", skiprows=1)

2. Create polygon from vertices using `shapely.geometry.Polygon`

These data have a bit of an odd structure (i.e., not a tidy dataframe with columns for coordinates) and so need a special helper function to properly order the coordinates.

In [3]:
def get_polygon_coords(row):
    """Create shapely polygon object from row of alternating
    lat/lon coords

    Args:
        pandas series object from df.iterrows() of MASIE regions
        MS Excel file

    returns
        shapely polygon derived from vertices coordinates in row

    Notes:
        designed for use with MASIE_regions_polygon_vertices.xls
    """
    df_di = {"lat": [], "lon": []}
    for idx, value in zip(row.index, row):

        if pd.isnull(value):
            df_di["lat"].append(df_di["lat"][0])
            df_di["lon"].append(df_di["lon"][0])
            break
        if "Lat" in idx:
            df_di["lat"].append(value)
        if "Lon" in idx:
            df_di["lon"].append(value)

    return list(zip(df_di["lon"], df_di["lat"]))

poly_coords = {index: get_polygon_coords(row) for index, row in masie_df.iterrows()}

print(f"Coordinate pairs for {masie_df['Name'][0]}:\n")
_ = [print(pair) for pair in poly_coords[masie_df.index[0]]]

Coordinate pairs for Beaufort Sea:

(-156.645452750466, 65.3734851649356)
(-156.479166999999, 79.9999999999988)
(-112.339999999999, 77.6899999999999)
(-124.575, 75.6699999999999)
(-123.999999999999, 64.9999999999993)
(-156.645452750466, 65.3734851649356)


In [4]:
# create accumulator dataframe from region number and name columns
masie_polys = masie_df[["Name"]].copy()

for index, pair_list in poly_coords.items():
    masie_polys.loc[index, "geometry"] = Polygon(pair_list)
    
masie_polys

,Name,geometry
0,Beaufort Sea,"POLYGON ((-156.645452750466 65.3734851649356, ..."
1,Chukchi Sea,"POLYGON ((-156.645461249268 65.373488287082, -..."
2,East Siberian Sea,"POLYGON ((145 64.9999999999993, 145 79.9999999..."
3,Laptev Sea,"POLYGON ((99.9999999999999 64.9999999999993, 9..."
4,Kara Sea,"POLYGON ((65 64.9999999999993, 53.664899999999..."
5,Barents Sea,"POLYGON ((34 59.9999999999973, 29.011 67.06399..."
6,Greenland Sea,"POLYGON ((-44.9999999999999 54.9999999999913, ..."
7,Baffin Bay/Gulf of St. Lawrence,"POLYGON ((-53.1999999999999 41.9999999999997, ..."
8,Canadian Archipelago,"POLYGON ((-103.411 60.6899999999977, -123.9999..."
9,Hudson Bay,"POLYGON ((-81.23630948167229 49.1949030383904,..."


## Writing to shapefile

The `geopandas` library makes this very easy:
1. make a GeoDataFrame from the DataFrame of Polygons 
2. set the CRS with the `set_crs()` method 
3. and write to disk with the `to_file()` method

In [5]:
masie_polys = gpd.GeoDataFrame(masie_polys).set_crs(epsg=4326).to_file("ancillary/shp/MASIE_regions_4326.shp")